In [ ]:
# ==========================================
# 1. SETUP & IMPORTS
# ==========================================
import os
import glob
import time
import cv2
import json
import sys
import pandas as pd
import numpy as np
import warnings
import subprocess
from collections import defaultdict
from tqdm import tqdm

warnings.simplefilter(action='ignore', category=FutureWarning)

# ==========================================
# 2. AUTHENTICATION & DATA VERIFICATION
# ==========================================
# We only auth if we actually need to download something.
KEY_FILE = 'colab-upload-bot-key.json'

# --- PATHS (Relative for Workbench) ---
LOCAL_BASE_DIR = './data_local'
LOCAL_TRAIN_DIR = os.path.join(LOCAL_BASE_DIR, 'trainsm')
LOCAL_JSON_PATH = './train.json'
OUTPUT_CSV_PATH = './metrics/strat_1_cpu.csv'

# GCS Config (Only used if local data is missing)
BUCKET_NAME = 'vis-data-2025'
GCS_TRAIN_DIR = f'gs://{BUCKET_NAME}/trainsm'
GCS_JSON_URL = f'gs://{BUCKET_NAME}/train.json'

print(f"\n🚀 CHECKING DATA...")

# A. Check Annotations
if not os.path.exists(LOCAL_JSON_PATH):
    print(f"⬇️ 'train.json' not found. Downloading...")
    if os.path.exists(KEY_FILE):
        os.system(f'gcloud auth activate-service-account --key-file="{KEY_FILE}"')
    os.system(f'gsutil cp {GCS_JSON_URL} {LOCAL_JSON_PATH}')
else:
    print("✅ Annotations found locally.")

# B. Check Video Data
if os.path.exists(LOCAL_TRAIN_DIR) and len(os.listdir(LOCAL_TRAIN_DIR)) > 0:
    print(f"✅ Training data found in '{LOCAL_TRAIN_DIR}'. Skipping download.")
else:
    print(f"⬇️ Data not found. Downloading from GCS...")
    if os.path.exists(KEY_FILE):
        os.system(f'gcloud auth activate-service-account --key-file="{KEY_FILE}"')
    
    os.makedirs(LOCAL_BASE_DIR, exist_ok=True)
    # Download with progress bar logic (Simplified for brevity)
    os.system(f'gsutil -m cp -r {GCS_TRAIN_DIR} {LOCAL_BASE_DIR}')

# ==========================================
# 3. MOTION COMPENSATION LOGIC (CPU)
# ==========================================
def align_frames(prev_gray, curr_gray):
    """
    Calculates camera motion and warps prev_gray to match curr_gray.
    """
    # 1. Detect Features (Shi-Tomasi Corners)
    # We limit to 200 points to keep CPU speed high
    prev_pts = cv2.goodFeaturesToTrack(prev_gray, maxCorners=200, qualityLevel=0.01, minDistance=30)

    if prev_pts is None: 
        return None # Cannot align

    # 2. Optical Flow (Lucas-Kanade) - Track points to current frame
    curr_pts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, prev_pts, None)

    # Filter valid points
    good_prev = prev_pts[status == 1]
    good_curr = curr_pts[status == 1]

    if len(good_prev) < 4: 
        return None # Need 4 points to calculate Homography

    # 3. Find Homography (Transformation Matrix)
    # RANSAC filters out outliers (like the moving bird itself!)
    H, mask = cv2.findHomography(good_prev, good_curr, cv2.RANSAC, 5.0)

    if H is None:
        return None

    # 4. Warp Previous Frame to match Current Frame
    height, width = prev_gray.shape
    warped_prev = cv2.warpPerspective(prev_gray, H, (width, height))

    return warped_prev

# ==========================================
# 4. HELPER FUNCTIONS
# ==========================================
def load_json_ground_truth(json_path):
    if not os.path.exists(json_path): return {}
    with open(json_path, 'r') as f: data = json.load(f)
    
    id_to_filename = {img['id']: img['file_name'] for img in data['images']}
    img_id_to_boxes = defaultdict(list)
    if 'annotations' in data:
        for ann in data['annotations']:
            img_id_to_boxes[ann['image_id']].append(ann['bbox'])

    filename_to_gt = {}
    for img_id, filename in id_to_filename.items():
        key = filename
        if key.startswith('train/'):
            key = key.replace('train/', '', 1)
        filename_to_gt[key] = img_id_to_boxes.get(img_id, [])
    
    return filename_to_gt

def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    xi1, yi1 = max(x1, x2), max(y1, y2)
    xi2, yi2 = min(x1 + w1, x2 + w2), min(y1 + h1, y2 + h2)
    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    union_area = (w1 * h1) + (w2 * h2) - inter_area
    return inter_area / union_area if union_area > 0 else 0

def get_next_version_path(path):
    """
    Returns a new file path with an incremented version number if the file already exists.
    Example: 'data.csv' -> 'data_1.csv' -> 'data_2.csv'
    """
    # If the file doesn't exist yet, simply return the original path
    if not os.path.exists(path):
        return path

    directory, filename = os.path.split(path)
    name, ext = os.path.splitext(filename)
    
    # Create the directory if it doesn't exist (safety check)
    if directory and not os.path.exists(directory):
        os.makedirs(directory)

    # Regex pattern to match files like "baseline_gpu_123.csv"
    # Matches: exact_name + underscore + digits + exact_extension
    pattern = re.compile(rf"^{re.escape(name)}_(\d+){re.escape(ext)}$")
    
    max_version = 0
    
    # List files in the directory to find the highest existing number
    for f in os.listdir(directory if directory else '.'):
        match = pattern.match(f)
        if match:
            version = int(match.group(1))
            if version > max_version:
                max_version = version

    # Next version is max found + 1
    new_filename = f"{name}_{max_version + 1}{ext}"
    return os.path.join(directory, new_filename)

# ==========================================
# 5. MAIN PIPELINE
# ==========================================
def run_gmc_evaluation():
    gt_data = load_json_ground_truth(LOCAL_JSON_PATH)
    if not gt_data: 
        print("❌ Annotations not loaded.")
        return
    
    start_time = time.time()

    # Find videos in ./data_local/trainsm/
    video_folders = sorted(glob.glob(os.path.join(LOCAL_TRAIN_DIR, '*')))
    video_folders = [f for f in video_folders if os.path.isdir(f)]

    if not video_folders:
        print(f"❌ No video folders found in {LOCAL_TRAIN_DIR}.")
        return

    print(f"📂 Found {len(video_folders)} videos. Starting GMC (CPU)...")
    print(f"\n{'Video':<10} | {'Frames':<6} | {'FPS':<6} | {'Prec':<6} | {'Recall':<6} | {'F1':<6}")
    print("-" * 65)

    total_tp = total_fp = total_fn = total_time = total_frames = 0
    results_data = []

    for video_path in video_folders:
        video_name = os.path.basename(video_path)
        images = sorted(glob.glob(os.path.join(video_path, '*.jpg')))
        if not images: continue

        vid_tp = vid_fp = vid_fn = 0
        vid_start = time.time()
        n_frames = len(images)
        
        # Initialize Previous Frame
        prev_gray = None

        for i, img_path in enumerate(images):
            # Print progress cleanly
            if i % 50 == 0:
                percent = ((i + 1) / n_frames) * 100
                sys.stdout.write(f"\r👉 Processing [{video_name}] Frame {i+1}/{n_frames} ({percent:.1f}%)")
                sys.stdout.flush()

            # 1. Load & Grayscale
            frame = cv2.imread(img_path)
            if frame is None: continue
            curr_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            preds = []
            
            # We need a previous frame to compare against
            if prev_gray is not None:
                # 2. Align Frames (GMC)
                warped_prev = align_frames(prev_gray, curr_gray)

                if warped_prev is not None:
                    # 3. Difference & Threshold
                    diff = cv2.absdiff(curr_gray, warped_prev)
                    _, thresh = cv2.threshold(diff, 60, 255, cv2.THRESH_BINARY)

                    # 4. Clean Noise
                    kernel = np.ones((3,3), np.uint8)
                    thresh = cv2.dilate(thresh, kernel, iterations=2)
                    thresh = cv2.erode(thresh, kernel, iterations=1)

                    # 5. Detect Contours (Blobs)
                    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                    
                    for cnt in contours:
                        area = cv2.contourArea(cnt)
                        # Filter: Ignore tiny noise (<20) and massive alignment errors (>2000)
                        if 100 < area < 2000: 
                            x, y, w, h = cv2.boundingRect(cnt)
                            
                            # 6. Border Filtering
                            # Warping creates black bars at edges. We ignore detections there.
                            h_img, w_img = curr_gray.shape
                            border = 15 
                            if x > border and y > border and (x+w) < (w_img-border) and (y+h) < (h_img-border):
                                preds.append([x, y, w, h])

            # Store current frame for next iteration
            prev_gray = curr_gray

            # 7. Evaluate vs Ground Truth
            key = f"{video_name}/{os.path.basename(img_path)}"
            gts = gt_data.get(key, [])
            matched_gt = set()

            for p_box in preds:
                best_iou = 0
                best_idx = -1
                for idx, g_box in enumerate(gts):
                    if idx in matched_gt: continue
                    iou = calculate_iou(p_box, g_box)
                    if iou > best_iou: best_iou = iou; best_idx = idx
                
                # IoU Threshold: Lower (0.20) for motion logic as shapes are imprecise
                if best_iou >= 0.20: 
                    vid_tp += 1
                    matched_gt.add(best_idx)
                else:
                    vid_fp += 1
            
            vid_fn += len(gts) - len(matched_gt)

        # End of Video Stats
        vid_time = time.time() - vid_start
        fps = len(images) / vid_time if vid_time > 0 else 0
        prec = vid_tp / (vid_tp + vid_fp) if (vid_tp + vid_fp) > 0 else 0
        rec = vid_tp / (vid_tp + vid_fn) if (vid_tp + vid_fn) > 0 else 0
        f1 = 2 * (prec * rec) / (prec + rec) if (prec + rec) > 0 else 0

        # Clear line
        sys.stdout.write("\r" + " " * 80 + "\r")
        print(f"{video_name:<10} | {len(images):<6} | {fps:<6.1f} | {prec:<6.2f} | {rec:<6.2f} | {f1:<6.2f}")

        results_data.append({
            'Video': video_name, 'Frames': len(images), 'FPS': round(fps, 2),
            'Precision': round(prec, 4), 'Recall': round(rec, 4), 'F1': round(f1, 4),
            'TP': vid_tp, 'FP': vid_fp, 'FN': vid_fn
        })

        total_time += vid_time; total_frames += len(images)
        total_tp += vid_tp; total_fp += vid_fp; total_fn += vid_fn

    # --- FINAL SUMMARY ---
    avg_fps = total_frames / total_time if total_time > 0 else 0
    overall_prec = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
    overall_rec = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
    overall_f1 = 2 * (overall_prec * overall_rec) / (overall_prec + overall_rec) if (overall_prec + overall_rec) > 0 else 0

    print("=" * 65)
    print("FINAL RESULTS (GMC / Motion Compensation):")
    print(f"Total Frames:   {total_frames}")
    print(f"Average FPS:    {avg_fps:.2f}")
    print(f"Precision:      {overall_prec:.4f}")
    print(f"Recall:         {overall_rec:.4f}")
    print(f"F1-Score:       {overall_f1:.4f}")
    print("=" * 65)

    df = pd.DataFrame(results_data)
    overall_row = {
        'Video': 'OVERALL', 'Frames': total_frames, 'FPS': round(avg_fps, 2),
        'Precision': round(overall_prec, 4), 'Recall': round(overall_rec, 4),
        'F1': round(overall_f1, 4), 'TP': total_tp, 'FP': total_fp, 'FN': total_fn
    }
    df = pd.concat([df, pd.DataFrame([overall_row])], ignore_index=True)
    final_path = get_next_version_path(OUTPUT_CSV_PATH)
    df.to_csv(final_path, index=False)
    print(f"✅ CSV Saved: {final_path}")
    elapsed_time = time.time() - start_time
    print(f"⏱️ Process took: {elapsed_time:.2f} seconds ({elapsed_time/60:.2f} minutes)")

if __name__ == "__main__":
    run_gmc_evaluation()